In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [5]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl import Workbook

# Быстрое решение

## Выгрузка отформатированных датафреймов в разные листы

In [15]:
df1 = pd.DataFrame({
    'date': [dt.date(2022, 1, 1), dt.date(2022, 1, 1), dt.date(2022, 1, 2), dt.date(2022, 1, 2), dt.date(2022, 1, 3)],
    'colA':[-3, -2, -1, -2, -3],
    'colB': [1, 2, 3, 4, 5],
    'colC': [1, 2, 3, 2, 1],
    'col1': ['A','B','C','D','E'],
    'col2': [1, 2, 3, 4, 5],
    'col3': [5, 4, 3, 2, 1]
})

df1_colnum = {key: value for value, key in enumerate(df1.columns)}
df1_widths = {col: len(col) + 8 for col in df1.columns}
borders = df1.reset_index().groupby('date').last()['index'].tolist()

df1_styled = (
    df1.style
        .format({col: "{}" for col in df1.columns})
        .background_gradient(subset=['col2','col3'], cmap='Greys')
        .bar(subset=['colA', 'colC'], color=['#EC7063', '#48C9B0'], axis=0, align='mid')
        .bar(subset=['colB'], color=['#CACFD2'], axis=0, align='left')
        .apply(lambda row: ['border-bottom-style: solid; border-bottom-width: 10px']*len(row) if row.name in borders else ['']*len(row), axis=1)
)
df1_styled

,date,colA,colB,colC,col1,col2,col3
0,2022-01-01,-3,1,1,A,1,5
1,2022-01-01,-2,2,2,B,2,4
2,2022-01-02,-1,3,3,C,3,3
3,2022-01-02,-2,4,2,D,4,2
4,2022-01-03,-3,5,1,E,5,1


Вывод внутренних barchart-ов на текущий момент (31.08.2022) не отображается в Excel.

In [17]:
df2 = pd.DataFrame({
    'col1': ['a','b','c'],
    'col2': [5,1,50],
    'col3': [4,2,40],
    'col4': [3,3,30],
    'col5': [2,4,20],
    'col6': [1,5,10]
})

df2_colnum = {key: value for value, key in enumerate(df2.columns)}
df2_widths = {col: len(col) + 8 for col in df2.columns}

df2_styled = (
    df2.style
        .format({col: "{}" for col in df2.columns})
        .background_gradient(subset=['col2','col3','col4','col5','col6'], axis=1, cmap='Greens')
)

df2_styled

,col1,col2,col3,col4,col5,col6
0,a,5,4,3,2,1
1,b,1,2,3,4,5
2,c,50,40,30,20,10


In [18]:
writer = pd.ExcelWriter('data/result/ex.xlsx', engine='openpyxl')
book = Workbook()

book.remove(book['Sheet'])

In [7]:
for sheetname, coldict, widths, df in zip(
    ['df1', 'df2'], 
    [df1_colnum, df2_colnum], 
    [df1_widths, df2_widths], 
    [df1_styled, df2_styled]):
    
    sheet = book.create_sheet(sheetname)
    print(sheet)
    
    for col in widths:
        sheet.column_dimensions[get_column_letter(coldict[col]+1)].width = widths[col]
    
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    
    df.to_excel(writer, sheet_name = sheetname, index=False)


writer.save()

<Worksheet "df1">
<Worksheet "df2">


### Комплексное решение

In [8]:
def replace_unnamed(df):
    unnamed = []
    for row in df.columns:
        for col in row:
            if 'Unnamed:' in col:
                unnamed.append(col)
    df = (df.rename(columns={key: '' for key in unnamed}))
    return df

In [9]:
def scale_columns(df, scale_level):
    '''Функция собирает словарь {колонка : ширина колонки как минимум из ширины содержания и ширины названия }'''
    widths = dict()
    for idx, col in enumerate(df):
        series = df[col]
        
        max_len = max(
            series.astype(str).map(len).max(),
            len(str(series.name)) + 5 if isinstance(series.name, str) else len(str(series.name[scale_level])) + 5)

        widths[col] = min([max_len, 80])
    return widths

In [10]:
def drop_header(df):
    '''Функция сбрасывает header и создает словарь {колонка : номер колонки}'''
    coldict = {col: num for col, num in zip(df.columns, range(len(df.columns)))}
    df = df.reset_index(drop=True)
    df.columns = range(df.shape[1])

    return df, coldict

In [11]:
def scale_columns(df, scale_level):
    '''Функция собирает словарь {колонка : ширина колонки как минимум из ширины содержания и ширины названия }'''
    widths = dict()
    for idx, col in enumerate(df):
        series = df[col]
        
        max_len = max(
            series.astype(str).map(len).max(),
            len(str(series.name)) + 5 if isinstance(series.name, str) else len(str(series.name[scale_level])) + 5)

        widths[col] = min([max_len, 80])
    return widths

### Настройка стиля (толщина границ, цвет фона и шрифта, выделение текста)

In [12]:
def set_attr(style, attr, indexes):
    ''' Функция добавляет новый атрибут к списку стилей.'''
    for ind in indexes:
        style[ind] += f'; {attr}'
    return style

def set_style_properties(df, styledict):
    '''
    Функция получает на вход таблицу, а также схему стилей в виде словаря. Пример схемы стилей
    
    styledict = {'borders': [{'attr': "border-bottom-style: solid; border-bottom-width: 10px",
                              'indexes': [2,4]},], # индексы строк к которым нужно применить этот стиль
                              
                 'backgrounds': [{'attr': f"background-color: {'green'}", 
                                  'indexes': [1,4]},], 
                                  
                 'colors': [{'attr': f"color: {'red'}", 
                             'indexes': [2]},], 
                             
                 'bolds': [{'attr': "font-weight: bold", 
                            'indexes': [1,3]},]}
                            
    Для каждой строки таблицы формируется список стилей (в словаре style), 
    которые применяются в конце. Функция возвращает объект Styler
    '''
    
    style = {ind: '' for ind in df.index}
    
    # borders
    if 'borders' in styledict:
        for border in styledict['borders']:
            style = set_attr(style, border['attr'], border['indexes'])
    
    # backgrounds
    if 'backgrounds' in styledict:
        for background in styledict['backgrounds']:
            style = set_attr(style, background['attr'], background['indexes'])
    
    # colors
    if 'colors' in styledict:
        for color in styledict['colors']:
            style = set_attr(style, color['attr'], color['indexes'])
    
    # bolds
    if 'bolds' in styledict:
        for bold in styledict['bolds']:
            style = set_attr(style, bold['attr'], bold['indexes'])
    
    
    df_styled = df.style.apply(lambda row: [style[row.name]]*len(row), axis=1)
    
    return df_styled
    
    


### Настройка листа (ширина, группировки, фильтрация)

In [13]:
def set_width(sheet, coldict, widths):
    '''Каждой колонке простовляется ширина из weights'''
    for col in widths:
        sheet.column_dimensions[get_column_letter(coldict[col]+1)].width = widths[col]
    return sheet

def set_cols_grouping(sheet, head, groups):
    '''Собираем колонки в группы, указанные в словаре groups'''
    for level in groups:
        for group, left in groups[level]:
            groupcols = [get_column_letter(col+1) for col in head[head[level] == group].index.tolist()[left:]]
            sheet.column_dimensions.group(groupcols[0],groupcols[-1], hidden=True)
    return sheet
def set_rows_grouping(sheet, groups):
    '''Собираем строки в группы, указанные в списке groups'''
    for group, left in groups:
        grouprows = group[left:]
        sheet.row_dimensions.group(grouprows[0], grouprows[-1], hidden=True)
    return sheet

def set_filter(sheet, row, conditions):
    """Функция добавляет фильтрацию к таблице"""
    cols = list(conditions.keys())
    sheet.auto_filter.ref = f"{get_column_letter(cols[0]+1)}{row}:{get_column_letter(cols[-1]+1)}{row}"
    
    for col in conditions:
        if conditions[col]:
            sheet.auto_filter.add_filter_column(col, conditions[col])
    
    return sheet
        

def set_writer_properties(sheet, 
                          coldict={}, widths={}, 
                          head={}, cols_grouping={}, 
                          rows_grouping={}, 
                          filtering=()):
    
    # widths
    if widths:
        sheet = set_width(sheet, coldict, widths)
    
    # cols grouping
    # проставляем, какие колонки стоит сгруппировать (предполагается, что группа объединена в шапку). 
    # На втором месте в картеже стоит число. Оно задает количество колонок, которые стоит оставить видимыми как заголовок группы
    if cols_grouping:
        sheet = set_cols_grouping(sheet, head, cols_grouping)
    
    # rows grouping
    if rows_grouping:
        sheet = set_rows_grouping(sheet, rows_grouping)
    
    # filtering
    if filtering:
        row, conditions = filtering
        sheet = set_filter(sheet, row, conditions)
    
    return sheet

